In [84]:
# Loading Libraries
import os
import csv
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error
from math import sqrt

# Part 6: Naive collaborative ﬁltering

In [2]:
# loading the data
file_path_ratings = os.path.expanduser('~/Documents/GitHub/ee219/Project 3/ml-latest-small/ml-latest-small/ratings.csv')
file_path_movies = os.path.expanduser('~/Documents/GitHub/ee219/Project 3/ml-latest-small/ml-latest-small/movies.csv')

movie_list = []

user_id = []
movie_id = []
ratings = []

# loading the movie list
with open(file_path_movies) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        movie_list.append(int(row['movieId']))

# loading the user_id and ratings
with open(file_path_ratings) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        user_id.append(int(row['userId']))
        movie_id.append(int(row['movieId']))
        ratings.append(float(row['rating']))

In [3]:
# Making the R matrix
# There are 671 users and 9125 movies
R = np.zeros((671, 9125))
for i in xrange(0, np.size(user_id)):
    x = user_id[i] - 1
    y = movie_list.index(movie_id[i])    
    R[x,y] = ratings[i]

## Question 30

In [28]:
# Finding the mean ratings for all users
mean_ratings = np.sum(R, 1) / np.size(R, 0)
data = []
data.append(user_id)
data.append(movie_id)
data.append(ratings)
data = np.transpose(data)

In [79]:
def naive_collab_filt(test):
    N = np.size(test,0)
    predicted = []
    actual = []
    for i in xrange(0, N, 1):
        predicted = mean_ratings[int(test[i][0] -1)] # Getting the mean rating from current user
        actual = test[i][2]
    
    return predicted, actual

def root_mean_square_error(actual, pred):
    return sqrt(np.nanmean(np.square(np.subtract(pred, actual))))

In [83]:
kf = KFold(n_splits=10)

RMSE = []

for train_index, test_index in kf.split(data):
    y_predicted, y_actual = naive_collab_filt(data[test_index])
    RMSE.append(root_mean_square_error(y_actual, y_predicted))
    
    
print 'Average Overall RMSE is ', np.nanmean(RMSE)

Average Overall RMSE is  3.44642324888


## Questions 31, 32, 33

In [87]:
# Finding the number of ratings per movie
count_m = np.count_nonzero(R, axis=0)
# var_m = variance of movies
var_m = np.var(R, axis = 0)
# movie_list has the list of movie_ids to correlate it with the position in array

In [88]:
## Testdata is split into 3 columns (user_id-string, movie_id-)
def trim(testdata):
    popular = []
    nonpopular = []
    high_var = []
    
    for i in xrange(0, np.size(testdata,0), 1):
        index = movie_list.index(int(testdata[i][1]) ) 
                                 
        # Testing for popular/nonpopular   
        if count_m[index] > 2:
            popular.append(testdata[i])
        else:
            nonpopular.append(testdata[i])
        
        # Testing for high variance
        if (count_m[index] >= 5) and (var_m[index] >= 2):
            high_var.append(testdata[i])
                                 
    
    return popular, nonpopular, high_var

In [89]:
# Making RMSE
RMSE_P = []
RMSE_NP = []
RMSE_HV = []

for train_index, test_index in kf.split(data):
    popular, nonpopular, high_var = trim(data[test_index])
    
    # Calculating popular
    y_predicted, y_actual = naive_collab_filt(popular)
    RMSE_P.append(root_mean_square_error(y_actual, y_predicted))
    
    # Calculating nonpopular
    y_predicted, y_actual = naive_collab_filt(nonpopular)
    RMSE_NP.append(root_mean_square_error(y_actual, y_predicted))
    
    # Calculating high variance
    y_predicted, y_actual = naive_collab_filt(high_var)
    RMSE_HV.append(root_mean_square_error(y_actual, y_predicted))
    
print 'Average RMSE for popular movies is ', np.nanmean(RMSE_P)
print 'Average RMSE for nonpopular movies is ', np.nanmean(RMSE_NP)
print 'Average RMSE for high variance movies is ', np.nanmean(RMSE_HV)

Average RMSE for popular movies is  3.39642324888
Average RMSE for nonpopular movies is  3.1629657228
Average RMSE for high variance movies is  3.33464977645
